# Bond Fundamentals

This notebook demonstrates how to access and use the **Bond** functionalities within the **LSEG Financial Analytics SDK**. 

Bond instruments are fixed income securities that represent debt obligations. Bond types supported:
- Fixed Rate Bonds: Bonds with fixed coupon payments over the life of the bond
- Floating Rate Bonds: Bonds with variable coupon rates tied to reference rates (SOFR, LIBOR, etc.)
- Zero Coupon Bonds: Bonds that pay no periodic interest, sold at discount to face value
- Inflation-Linked Bonds: Bonds with principal and/or coupons adjusted for inflation
- Callable/Puttable Bonds: Bonds with embedded options for early redemption

**You will be able to:**
- Define Bond instruments with basic parameters (coupon, maturity, face value)
- Configure pricing parameters and market data settings
- Evaluate Bond Analytics (price, yield, duration, convexity)

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.bond` - for Bond instrument definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import bond as bd

import pandas as pd
import json
import datetime as dt
from IPython.display import display

## Data Preparation

To define a Bond instrument you need to follow a structured 3-step process:

1. **Bond Definition** - Specify basic bond parameters (coupon rate, maturity date, face value, payment frequency)
2. **Bond Instrument Definition** - Create the instrument object for pricing
3. **Pricing Preferences** - Configure pricing parameters - optional

### Bond Definition

Two methods to define a bond are possible:

- Using **fully user defined instrument**
- Using an existing **Market Instrument Code**

#### User-defined bonds

To built correctly a user-defined Bond, the following structure components can be included

**Core Parameters:**
- **Face Value/Par Value**: Principal amount paid at maturity
- **Coupon Rate**: Annual interest rate paid on the face value
- **Maturity Date**: When the bond expires and principal is repaid
- **Payment Frequency**: How often coupons are paid (annual, semi-annual, quarterly, etc.)

**Market Conventions:**
- **Day Count Method**: How interest accrual is calculated (30/360, Actual/360, Actual/365, etc.)
- **Business Day Conventions**: How payment dates are adjusted for holidays
- **Settlement Convention**: Standard settlement period (T+1, T+2, T+3)

**Analytics Available:**
- **Price**: Clean price, dirty price (including accrued interest)
- **Yield**: Yield to maturity, current yield, yield to call
- **Duration**: Modified duration, Macaulay duration
- **Convexity**: Second-order price sensitivity to interest rate changes
- **Spread**: Credit spread over benchmark curve

In this notebook we define three types of user-defined bonds:
- Fixed rate Bond
- Floating rate Bond
- Zero-coupon Bond

In [2]:
# Example 1: Fixed rate bond definition

bond_definition_fixed_rate_user_defined = bd.BondDefinition(
    # Core Bond Parameters
        # Face value/Par value
    notional_amount=1_000_000,                                 # $1M face value (default: 1,000,000)
    notional_ccy="USD",                                        # USD currency
    
        # Coupon Rate and Type
    fixed_rate_percent=4.25,                                   # 4.25% annual coupon rate
    interest_type=bd.InterestTypeEnum.FIXED,                   # Fixed rate bond. Options: FIXED, FLOAT
    
        # Bond Timing
    issue_date=dt.datetime(2020, 3, 15),                       # Bond issue date
    end_date=dt.datetime(2025, 3, 15),                         # 5-year maturity in date format (Or use tenor directly instead of end_date: )                            
    
        # Payment Structure
    interest_payment_frequency=bd.InterestPaymentFrequencyEnum.SEMI_ANNUAL,  # Semi-annual coupons. Options: ANNUAL, SEMI_ANNUAL, QUARTERLY, MONTHLY, etc.
    first_regular_payment_date=dt.datetime(2020, 9, 15),      # First coupon payment date
    
    # Market Conventions
    interest_calculation_method=bd.InterestCalculationMethodEnum.DCB_30_360,  # 30/360 day count. Options: DCB_30_360, DCB_ACTUAL_360, DCB_ACTUAL_365, etc.
    payment_business_day_convention=bd.PaymentBusinessDayConventionEnum.MODIFIED_FOLLOWING,  # Business day adjustment. Options: MODIFIED_FOLLOWING, NEXT_BUSINESS_DAY, PREVIOUS_BUSINESS_DAY
    payment_business_days="USA",                               # US holiday calendar
    
    # Optional Parameters
    instrument_tag="USD_5Y_BOND_4.25",                         # User-defined identifier (max 40 chars)
    stub_rule=bd.FinancialContractStubRuleEnum.MATURITY,       # Stub period alignment. Options: ISSUE, MATURITY
    is_perpetual=False,                                        # Non-perpetual bond
)


# Display comprehensive information about the created bond
print(f"✓ Fixed rate Bond:")
print(f"   Bond Type: {bond_definition_fixed_rate_user_defined.interest_type}")
print(f"   Notional Amount: {bond_definition_fixed_rate_user_defined.notional_ccy} {bond_definition_fixed_rate_user_defined.notional_amount:,.0f}")
print(f"   Coupon Rate: {bond_definition_fixed_rate_user_defined.fixed_rate_percent}%")
print(f"   Payment Frequency: {bond_definition_fixed_rate_user_defined.interest_payment_frequency}")
print(f"   Issue Date: {bond_definition_fixed_rate_user_defined.issue_date.strftime('%Y-%m-%d')}")
print(f"   Maturity Date: {bond_definition_fixed_rate_user_defined.end_date.strftime('%Y-%m-%d')}")
print(f"   Day Count: {bond_definition_fixed_rate_user_defined.interest_calculation_method}")
print(f"   Business Day Convention: {bond_definition_fixed_rate_user_defined.payment_business_day_convention}")
print(f"   Calendar: {bond_definition_fixed_rate_user_defined.payment_business_days}")
print(f"   Instrument Tag: {bond_definition_fixed_rate_user_defined.instrument_tag}")
print()

✓ Fixed rate Bond:
   Bond Type: InterestTypeEnum.FIXED
   Notional Amount: USD 1,000,000
   Coupon Rate: 4.25%
   Payment Frequency: InterestPaymentFrequencyEnum.SEMI_ANNUAL
   Issue Date: 2020-03-15
   Maturity Date: 2025-03-15
   Day Count: InterestCalculationMethodEnum.DCB_30_360
   Business Day Convention: PaymentBusinessDayConventionEnum.MODIFIED_FOLLOWING
   Calendar: USA
   Instrument Tag: USD_5Y_BOND_4.25



In [3]:
# Example 2: Floating rate bond definition

floating_bond = bd.BondDefinition(
    notional_amount=1_000_000,
    notional_ccy="USD",
    interest_type=bd.InterestTypeEnum.FLOAT,                    # FLOAT instead of FIXED
    
    # Floating Rate Parameters
    index_fixing_ric="USDSOFR=",                                # Reference rate RIC
    spread_bp=150,                                              # 150bp spread over SOFR
    index_reset_frequency=bd.IndexResetFrequencyEnum.MONTHLY,   # Monthly resets

    issue_date=dt.datetime(2023, 3, 15),
    end_date=dt.datetime(2033, 3, 15),
    interest_payment_frequency=bd.InterestPaymentFrequencyEnum.QUARTERLY,
    interest_calculation_method=bd.InterestCalculationMethodEnum.DCB_30_360,

    instrument_tag="USD_10Y_BOND_SOFR_150",
)

# Display comprehensive information about the created bond
print(f"✓ Floating rate Bond:")
print(f"   Bond Type: {floating_bond.interest_type}")
print(f"   Notional Amount: {floating_bond.notional_ccy} {floating_bond.notional_amount:,.0f}")
print(f"   Floating interest rate index: {floating_bond.index_fixing_ric}")
print(f"   Spread in addition to floating rate: {floating_bond.spread_bp}bps")
print(f"   Payment Frequency: {floating_bond.interest_payment_frequency}")
print(f"   Issue Date: {floating_bond.issue_date.strftime('%Y-%m-%d')}")
print(f"   Maturity Date: {floating_bond.end_date.strftime('%Y-%m-%d')}")
print(f"   Day Count: {floating_bond.interest_calculation_method}")
print(f"   Instrument Tag: {floating_bond.instrument_tag}")

✓ Floating rate Bond:
   Bond Type: InterestTypeEnum.FLOAT
   Notional Amount: USD 1,000,000
   Floating interest rate index: USDSOFR=
   Spread in addition to floating rate: 150.0bps
   Payment Frequency: InterestPaymentFrequencyEnum.QUARTERLY
   Issue Date: 2023-03-15
   Maturity Date: 2033-03-15
   Day Count: InterestCalculationMethodEnum.DCB_30_360
   Instrument Tag: USD_10Y_BOND_SOFR_150


In [4]:
# Example 3: Zero coupon bond definition

zero_coupon_bond = bd.BondDefinition(
    notional_amount=1_000_000,
    notional_ccy="USD",
    fixed_rate_percent=0.0,                                     # No coupon payments
    interest_payment_frequency=bd.InterestPaymentFrequencyEnum.ZERO,  # Zero frequency
    interest_calculation_method=bd.InterestCalculationMethodEnum.DCB_30_360,
    
    issue_date=dt.datetime(2020, 3, 15),
    end_date=dt.datetime(2025, 3, 15),

    instrument_tag="USD_5Y_BOND_ZC"
)

# Display comprehensive information about the created bond
print(f"✓ Zero Coupon Bond:")
print(f"   Notional Amount: {zero_coupon_bond.notional_ccy} {zero_coupon_bond.notional_amount:,.0f}")
print(f"   Coupon Rate: {zero_coupon_bond.fixed_rate_percent}%")
print(f"   Payment Frequency: {zero_coupon_bond.interest_payment_frequency}")
print(f"   Issue Date: {zero_coupon_bond.issue_date.strftime('%Y-%m-%d')}")
print(f"   Maturity Date: {zero_coupon_bond.end_date.strftime('%Y-%m-%d')}")
print(f"   Day Count: {zero_coupon_bond.interest_calculation_method}")
print(f"   Business Day Convention: {zero_coupon_bond.payment_business_day_convention}")
print(f"   Calendar: {zero_coupon_bond.payment_business_days}")
print(f"   Instrument Tag: {zero_coupon_bond.instrument_tag}")

✓ Zero Coupon Bond:
   Notional Amount: USD 1,000,000
   Coupon Rate: 0.0%
   Payment Frequency: InterestPaymentFrequencyEnum.ZERO
   Issue Date: 2020-03-15
   Maturity Date: 2025-03-15
   Day Count: InterestCalculationMethodEnum.DCB_30_360
   Business Day Convention: None
   Calendar: None
   Instrument Tag: USD_5Y_BOND_ZC


#### Bonds from Market instrument code

To define such a bond, it suffices to enter its instrument code (ISIN, RIC or CUSIP) and to include an instrument tag in format "BOND_{instrument_code_type}={instrument_code}" like below

In [5]:
# Method 2: Definition Using existing market instruments 

bond_definition_isin = bd.BondDefinition(
    instrument_code="US912810RQ31",   # ISIN, RIC, CUSIP
    instrument_tag="BOND_ISIN=US912810RQ31")

### Bond Instrument Definition

For each example defined below, we create a bond instrument.

In [6]:
bond_instrument1 = bd.BondDefinitionInstrument(definition=bond_definition_fixed_rate_user_defined)
bond_instrument2 = bd.BondDefinitionInstrument(definition=floating_bond)
bond_instrument3 = bd.BondDefinitionInstrument(definition=zero_coupon_bond)
bond_instrument4 = bd.BondDefinitionInstrument(definition=bond_definition_isin)

print("   Bond instrument containers created for pricing")

   Bond instrument containers created for pricing


### Bond Pricing Preferences

#### Setup pricing parameters

In [7]:
pricing_params = bd.BondPricingParameters(
    valuation_date=dt.datetime(2023, 7, 18),                   # Pricing date
    price_side=bd.PriceSideEnum.MID,                           # Use mid-market prices. Options: BID, ASK, MID, LAST
    report_ccy="USD",                                          # Reporting currency for analytics
    settlement_convention="2WD",                               # T+2 settlement convention
    
    # Optional pricing inputs (comment out to use market data)
    # yield_percent=4.50,                                      # Input yield to calculate price
    # clean_price=98.50,                                       # Input price to calculate yield
)

print(f"   Valuation Date: {pricing_params.valuation_date.strftime('%Y-%m-%d')}")
print(f"   Price Side: {pricing_params.price_side}")
print(f"   Report Currency: {pricing_params.report_ccy}")
print(f"   Settlement Convention: {pricing_params.settlement_convention}")

   Valuation Date: 2023-07-18
   Price Side: PriceSideEnum.MID
   Report Currency: USD
   Settlement Convention: 2WD


#### Bond Fields Setup

We select the below the following fields that are interesting for Bond pricing:

- **InstrumentTag**: Tag for identification
- **Notional currency**: Currency of the Bond
- **Market Value**: Present value of the Bond, it is also the **Dirty Price** of the bond
- **Report currency**: Currency of Bond valuation and metrics
- **Clean Price**: Bond price excluding accrued interest
- **Yield**: Yield to maturity of the Bond
- **Modified Duration**: Interest rate sensitivity measure
- **Convexity**: Second-order price sensitivity to yield changes
- **DV01**: Sensitivity of the market value to a 1bp parallel shift in the zero-coupon curve

In [8]:
# setup response fields

basic_fields = "InstrumentTag, NotionalCcy, MarketValueInDealCcy, ReportCcy, MarketValueInReportCcy"
additional_fields = "cleanPricePercent, dirtyPricePercent, yieldPercent, ModifiedDuration, convexity, DV01Bp"
fields = basic_fields + "," + additional_fields

## Request Execution

In [9]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instrument definitions for batch processing

try:
    response = bd.price(
        definitions=[bond_instrument1, bond_instrument2, bond_instrument3, bond_instrument4],
        pricing_preferences=pricing_params,
        fields=fields
    )

    # Display response structure information
    analytics_data = response['data']['analytics'][0]
    if analytics_data['error'] == {}:
        print("   Calculation successful!")
    else:
        print(f"   Pricing error: {analytics_data['error']}")
        
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise


   Calculation successful!


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Bond instrument setup (coupon rate, maturity date, notional amount, payment frequency), using the BondDefinition that we created.

 - **pricingPreferences**: Valuation date, price side (BID/ASK/MID/LAST), currency settings, settlement conventions.

 - **analytics**:
   - **tabularData**: Customizable list of analytics given in `fields` argument of price() function (`data`, `headers`, `statuses`)
   - **description**: Bond instrument summary and default fields used in the calculation
   - **nominalMeasures**: Main risk metrics calculated on Bond (`modifiedDuration`, `convexity`, `dv01Bp`)
   - **pricingAnalysis**: detailed price information on Bonds (`cleanPricePercent`, `dirtyPricePercent`, `yieldPercent`)
   - **valuation**: Detailed valuation of the Bond instru;ent with the below fields:
      - `MarketValueInDealCcy`: Present value of the Bond in deal currency
      - `MarketValueInReportCcy`: Present value of the Bond in report currency
   - **error**: Root cause of any issues, if encountered


We give below for the first deal the valuation details, as well as for the four deals the tabular data details, that include information on valuation and risk metrics

In [10]:
# Access the valuation object
valuation = response.data.analytics[0].valuation
print("Bond Valuation Results:")
print(json.dumps(valuation.as_dict(), indent=4))

Bond Valuation Results:
{
    "marketValueInDealCcy": 1066952.21880026,
    "marketValueInReportCcy": 1066952.21880026
}


In [11]:
# Convert multi-bond analytics to DataFrame for better visualization
print("Multi-Bond Analytics Summary:")
columns = [item['name'] for item in response['data']['analytics'][0]['tabularData']['headers']]
response_data = [item['tabularData']['data'] for item in response['data']['analytics']]

response_df = pd.DataFrame(response_data, columns=columns)

# Set InstrumentTag as index for better readability
response_df.set_index('InstrumentTag', inplace=True)

# Round numerical values to 4 decimal places while keeping strings unchanged
response_df_rounded = response_df.copy()
for col in response_df_rounded.columns:
    if response_df_rounded[col].dtype in ['float64', 'int64']:
        response_df_rounded[col] = response_df_rounded[col].round(4)

display(response_df_rounded.T)

Multi-Bond Analytics Summary:


InstrumentTag,USD_5Y_BOND_4.25,USD_10Y_BOND_SOFR_150,USD_5Y_BOND_ZC,BOND_ISIN=US912810RQ31
NotionalCcy,USD,USD,USD,USD
MarketValueInDealCcy,1066952.2188,1520285.2015,983634.4299,771035.0483
ReportCcy,USD,USD,USD,USD
MarketValueInReportCcy,1066952.2188,1520285.2015,983634.4299,771035.0483
CleanPricePercent,105.2431,151.4272,98.3634,76.0469
DirtyPricePercent,106.6952,152.0285,98.3634,77.1035
YieldPercent,1.0,0.8509,1.0,4.1435
ModifiedDuration,1.5915,0.1583,1.6419,15.9121
Convexity,3.3894,9.7069,4.3215,321.3819
DV01Bp,1.6981,2.8534,1.615,12.2688


## Appendix: Complete Functions Reference

The following is a complete list of all classes, enums, and functions available after importing from `lseg_analytics.pricing.instruments.bond`:

**Core Classes:**
- `BondDefinition` - Define bond instrument specifications (coupon, maturity, face value, payment frequency, conventions, etc.)
- `BondDefinitionInstrument` - Instrument container for pricing operations 
- `BondPricingParameters` - Configure pricing parameters
- `AmortizationItemDefinition` - Define amortization schedules for amortizing bonds

**Response Classes:**
- `BondCalculationResponse` - Main response container
- `BondValuation` - Individual bond valuation results
- `BondAnalyticsResponseData` - Detailed analytics and calculations
- `BondAnalyticsResponseWithError`: Analytics response with error
- `BondCashflows` - Detailed cashflow schedules
- `BondDescription` - Bond instrument summary information
- `BondPricingAnalysis` - Comprehensive pricing analysis results

**Bond Analytics Classes:**
- `BondNominalMeasures` - Different Bond risk measures (duration, DV01, convexity, etc.)
- `BondEffectiveMeasures` - Different Callable/Puttable Bond risk measures (duration, DV01, convexity, etc.)
- `BondSpreadMeasures` - Credit spread and basis point analytics
- `BondRoundingParameters` - Precision settings for calculations

**Bond Configuration Enums:**
- `InterestTypeEnum` - Bond type: FIXED, FLOAT, STEPPED
- `InterestPaymentFrequencyEnum` - Payment frequencies: ANNUAL, SEMI_ANNUAL, QUARTERLY, MONTHLY, ZERO, etc.
- `InterestCalculationMethodEnum` - Day count conventions: DCB_30_360, DCB_ACTUAL_360, DCB_ACTUAL_365, etc.
- `PaymentBusinessDayConventionEnum` - Business day adjustment: MODIFIED_FOLLOWING, NEXT_BUSINESS_DAY, etc.
- `FinancialContractStubRuleEnum` - Stub period alignment: ISSUE, MATURITY, etc.

**Floating Rate Bond Enums:**
- `IndexResetFrequencyEnum` - Reset frequencies: EVERYDAY, MONTHLY, QUARTERLY
- `ProjectedIndexCalculationMethodEnum` - Forward index calculation methods: CONSTANT_COUPON_PAYMENT, CONSTANT_INDEX, FORWARD_INDEX
- `IndexCompoundingMethodEnum` - Index compounding methodologies: AVERAGE, COMPOUNDED, etc.
- `IndexAverageMethodEnum` - Index averaging techniques: ARITHMETIC_AVERAGE, COMPOUNDED_ACTUAL, COMPOUNDED_AVERAGE_RATE, DAILY_COMPOUNDED_AVERAGE

**Market Data Enums:**
- `PriceSideEnum` - Market price sides: BID, ASK, MID, LAST
- `FxPriceSideEnum` - FX price side selection for currency conversion: BID, ASK, MID, LAST
- `IPAIndexObservationMethodEnum` - Index observation methods: LOOKBACK, MIXED, PERIOD_SHIFT

**Pricing Configuration Enums:**
- `QuotationModeEnum` - Bond quotation methods: CASH_GROSS_PRICE , YIELD, SPREAD, etc.
- `QuoteFallbackLogicEnum` - Fallback logic of bond quotation: BEST_FIELD, NONE
- `YieldTypeEnum` - Yield calculation types: ANNUAL_EQUIVALENT, DISCOUNT_ACTUAL_360, etc.
- `CreditSpreadTypeEnum` - Credit spread measurement types: FLAT_SPREAD, TERM_STRUCTURE
- `BenchmarkYieldSelectionModeEnum` - Benchmark curve selection methods: INTERPOLATE, NEAREST

**Rounding Configuration Enums:**
- `PriceRoundingEnum` / `PriceRoundingTypeEnum` - Price precision settings: ZERO to EIGHT + DEFAULT and UNROUNDED / CEIL, DOWN, etc.
- `YieldRoundingEnum` / `YieldRoundingTypeEnum` - Yield precision settings: ZERO to EIGHT + DEFAULT and UNROUNDED / CEIL, DOWN, etc.
- `SpreadRoundingEnum` / `SpreadRoundingTypeEnum` - Spread precision settings: ZERO to EIGHT + DEFAULT and UNROUNDED / CEIL, DOWN, etc.
- `AccruedRoundingEnum` / `AccruedRoundingTypeEnum` - Accrued interest precision: ZERO to EIGHT + DEFAULT and UNROUNDED / CEIL, DOWN, etc.

**Bond Structure Enums:**
- `AmortizationFrequencyEnum` - Amortization payment frequencies: ANNUAL, SEMI_ANNUAL, QUARTERLY, MONTHLY, ZERO, etc.
- `IPAAmortizationTypeEnum` - Amortization schedule types: ANNUITY, LINEAR, NONE, SCHEDULE
- `RedemptionDateTypeEnum` - Early redemption date handling: NATIVE_REDEMPTION_DATE, REDEMPTION_AT_CUSTOM_DAT, etc.
- `AdjustInterestToPaymentDateEnum` - Interest payment date adjustments: ADJUSTED, UNADJUSTED

**Market Data Classes:**
- `MarketDataQps` - Market data configuration and sources
- `DefaultBondQuote` - Default bond pricing quotes
- `HullWhiteParameters` - Hull & White interest rate model parameters

**Utility Enums:**
- `PaymentRollConventionEnum` - Payment date rolling conventions: LAST, LAST28, SAME, SAME1, SAME28
- `AccruedCalculationMethodEnum` - Accrued interest calculation methods: DCB_30_360, DCB_ACTUAL_360, DCB_ACTUAL_365, etc.
- `InflationModeEnum` - Inflation adjustment modes: ADJUSTED, UNADJUSTED, DEFAULT
- `VolatilityTermStructureTypeEnum` - Volatility surface types: HISTORICAL, IMPLIED
- `IPADirectionEnum` - Trade direction: PAID, RECEIVED
- `IPAVolatilityTypeEnum` - Volatility input types: FLAT, TERM_STRUCTURE
- `IPADividendTypeEnum` - Dividend input types: IMPLIED_TABLE, FORECAST_YIELD

**Main Functions:**
- `price()` - Execute bond pricing calculations with comprehensive analytics
- `functions_all()` - List all available functions in the bond module